In [131]:
# imports and setup 
from bs4 import BeautifulSoup
# you can use either of these libraries to get html from a website
import requests
import urllib.request

from ics import Calendar, Event
from collections import namedtuple
from datetime import datetime, time, date, timedelta

semester_start = "01/07/2019"
semester_end = "04/23/2019"
tz_offset = "7"
url = 'https://student.apps.utah.edu/uofu/stu/ClassSchedules/main/1194/class_list.html?subject=CS#'

In [115]:
ClassInfo = namedtuple('ClassInfo', 
                       ['number', 
                        'section', 
                        'component', 
                        'title', 
                        'days', 
                        'time', 
                        'location', 
                        'instructor'])

def generate_class_info(columns):
    time_day = cols[7].text.strip().split(" / ")
    if "TBA" in time_day:
        return None
    
    if '\n' in time_day[1]:
        time = time_day[1].split()[0]
    else:
        time = time_day[1]
        
    if '\n' in cols[10].text.strip():
        split_instructor = cols[10].text.strip().split('\n                                    \n                                    \n                                        ')
        instructor = ";".join(split_instructor)
    else:
        instructor = cols[10].text.strip()
        
    
    return ClassInfo(cols[2].text.strip(),
                    cols[3].text.strip(),
                    cols[4].text.strip(),
                    cols[6].text.strip(),
                    time_day[0],
                    time,
                    cols[8].text.strip(),
                    instructor)

In [116]:
classes = []

with urllib.request.urlopen(url) as response:
    html = response.read().decode('utf-8')

    bs = BeautifulSoup(html, 'html5lib')
    table = bs.find(id='classDetailsTable')
    # print(table)
    
    table_body = table.find('tbody')
    # print(table_body)
    
    rows = table.find_all('tr')
    for row in rows[1:]:
        if "notes" not in row.attrs['class']:
            cols = row.find_all('td')
            classes.append(generate_class_info(cols))
            
print(classes)

[ClassInfo(number='1001', section='001', component='Lecture', title='Eng Computing MATLAB', days='MoWe', time='03:00PM-04:20PM', location='WEB L110', instructor='DE ST GERMAIN, J. D.'), ClassInfo(number='1001', section='002', component='Laboratory', title='Eng Computing MATLAB', days='Fr', time='08:35AM-09:25AM', location='WEB L226', instructor='DE ST GERMAIN, J. D.'), ClassInfo(number='1030', section='001', component='Lecture', title='Foundations of CS', days='MoWe', time='02:00PM-02:50PM', location='WEB L101', instructor='JENSEN, P. A.'), ClassInfo(number='1030', section='002', component='Laboratory', title='Foundations of CS', days='TuTh', time='12:55PM-01:45PM', location='WEB L124', instructor='JENSEN, P. A.'), ClassInfo(number='1030', section='003', component='Laboratory', title='Foundations of CS', days='TuTh', time='02:00PM-02:50PM', location='WEB L124', instructor='JENSEN, P. A.'), ClassInfo(number='1030', section='004', component='Laboratory', title='Foundations of CS', days='

In [175]:
c = Calendar()
current_day = datetime.strptime(semester_start, "%m/%d/%Y")
semester_end_day = datetime.strptime(semester_end, "%m/%d/%Y")

def parse_days(days):
    ret = []
    n = 2
    
    for day in [days[i:i+n] for i in range(0, len(days), n)]:
        if day == "Mo":
            ret.append(1)
        elif day == "Tu":
            ret.append(2)
        elif day == "We":
            ret.append(3)
        elif day == "Th":
            ret.append(4)
        elif day == "Fr":
            ret.append(5)
            
    return ret

def parse_time(time):
    return time.split("-")   
        
    
def generate_time_delta(time):
    hours = int(time[0:2]) + (12 if time[-2:] == "PM" and time[0:2] != "12" else 0)
    minutes = int(time[3:5])
    return timedelta(hours=hours,minutes=minutes)
    
def generate_event(class_info: ClassInfo, date: datetime):
    start_time = parse_time(class_info.time)[0]
    end_time = parse_time(class_info.time)[1]
    
    stime = date + generate_time_delta(start_time)
    etime = date + generate_time_delta(end_time)
    
    e = Event()
    e.name = f"CS {class_info.number}-{class_info.section} {class_info.component}"
    e.begin = stime.strftime("%Y%m%dT%H:%M:%S")
    e.end = etime.strftime("%Y%m%dT%H:%M:%S")
    e.location = class_info.location
    e.description = f"Taught by {class_info.instructor}"
    
    return e

# print(generate_event(classes[0], semester_start_day))

def generate_calendar(classes, current_day, semester_end_day):
    c = Calendar()
    events = []
    
    while((semester_end_day - current_day).days >= 0):
        cur_day_of_week = current_day.isoweekday()
        for class_info in classes:
            if class_info is None:
                continue
                
            if cur_day_of_week in parse_days(class_info.days):
                events.append(generate_event(class_info, current_day))
                
        current_day += timedelta(days=1)
        
    c.events = events
    return c
    
c = generate_calendar(classes, current_day, semester_end_day)
print(str(c).replace("DTSTART", "DTSTART;TZID=America/Denver").replace("DTEND", "DTEND;TZID=America/Denver"))
with open("cs_class_schedule.ics", "w") as file:
    file.writelines(str(c).replace("DTSTART", "DTSTART;TZID=America/Denver").replace("DTEND", "DTEND;TZID=America/Denver"))

BEGIN:VCALENDAR
PRODID:ics.py - http://git.io/lLljaA
VERSION:2.0
BEGIN:VEVENT
DTSTAMP:20190115T052737Z
DTSTART;TZID=America/Denver:20190107T150000Z
DTEND;TZID=America/Denver:20190107T162000Z
SUMMARY:CS 1001-001 Lecture
DESCRIPTION:Taught by DE ST GERMAIN\, J. D.
LOCATION:WEB L110
TRANSP:OPAQUE
UID:fa7537c3-e446-4b46-9955-9e951e026509@fa75.org
END:VEVENT
BEGIN:VEVENT
DTSTAMP:20190115T052737Z
DTSTART;TZID=America/Denver:20190107T140000Z
DTEND;TZID=America/Denver:20190107T145000Z
SUMMARY:CS 1030-001 Lecture
DESCRIPTION:Taught by JENSEN\, P. A.
LOCATION:WEB L101
TRANSP:OPAQUE
UID:926dfaa7-765d-4dbc-8bab-31afeb12e054@926d.org
END:VEVENT
BEGIN:VEVENT
DTSTAMP:20190115T052737Z
DTSTART;TZID=America/Denver:20190107T104500Z
DTEND;TZID=America/Denver:20190107T113500Z
SUMMARY:CS 1410-001 Lecture
DESCRIPTION:Taught by JOHNSON\, D. E.
LOCATION:GC 1900
TRANSP:OPAQUE
UID:4dc16c9a-7e36-4585-b1ab-616ae3a8f568@4dc1.org
END:VEVENT
BEGIN:VEVENT
DTSTAMP:20190115T052737Z
DTSTART;TZID=America/Denver:20190107T1